# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm
import scipy.stats as stats

In [2]:
# --- Response ---

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.5, tt_var=0.2, tt_lb=0, tt_ub=1)
sse.load_A("Aest.csv")


/Users/jinningwang/Documents/work/andes/jwang/notes/ev_ssm.py:448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ev['na'][self.ev['na'] > self.ev['nam']] = self.ev['nam'][self.ev['na'] > self.ev['nam']]
/Users/jinningwang/Documents/work/andes/jwang/notes/ev_ssm.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ev['lc'][self.ev['na'] >= self.ev['nam']] = 1
EVA: ts=18[H], 10000 EVs, Total Q=87.83 MWh
Online 2955, Q=73.98 MWh, SoC=0.7712
Power(MW): Pt=-8.1388, Pc=-8.1388, Pd=-0.0
Ctrl: 0=1581; 1=1374; 
EVA: Load A from Aest.csv.


In [3]:
%run -i 'cosim_import.py'

0.post4472.dev0+ge6d2e42
2.8.0


In [4]:
caseH = 18
%run -i 'cosim_setup_andes.py'

EVA: ts=18[H], 50000 EVs, Total Q=433.49 MWh
Online 14439, Q=361.0 MWh, SoC=0.7685
Power(MW): Pt=-39.7925, Pc=-39.7925, Pd=-0.0
Ctrl: 0=7733; 1=6706; 
EVA: Load A from Aest.csv.
Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1449 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/ieee39_ev2.xlsx"...
Input file parsed in 0.0695 seconds.
System internal structure set up in 0.0233 seconds.


In [5]:
%run -i 'cosim_loadsyn.py'

ANDES total load is: 58.564 p.u.


In [6]:
%run -i 'cosim_setup_dispatch.py'

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0921 seconds.
0: |F(x)| = 7.561234474
1: |F(x)| = 2.917096747
2: |F(x)| = 0.1313034816
3: |F(x)| = 0.0003572030639
4: |F(x)| = 4.286541788e-09
Converged in 5 iterations in 0.0047 seconds.
Power flow results are consistent. Conversion is successful.


Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0037 seconds.
0: |F(x)| = 7.561234474
1: |F(x)| = 2.917096747
2: |F(x)| = 0.1313034816
3: |F(x)| = 0.0003572030639
4: |F(x)| = 4.286541788e-09
Converged in 5 iterations in 0.0038 seconds.
Power flow results are consistent. Conversion is successful.
GSF is consistent.
RTED GB model is loaded.
RTED removed: ['rampu', 'rampd']
RTED limited: sfru, sfrd
RTED: total cost=4910.262


pp cost=2224.7878007021177, gb cost=4910.261740428771


In [7]:
%run -i 'cosim_linktable.py'

In [8]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 3600

rru, rrd, rsfr = 1.0, 1.0, 0.04

# Define functions and constants used in loop
%run -i 'cosim_const.py'

Revise the EV control logic as: C->I, I->D, SOC high to low

In [9]:
# --- Response ---

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")

EVA: ts=18[H], 10000 EVs, Total Q=87.83 MWh
Online 2955, Q=73.98 MWh, SoC=0.7712
Power(MW): Pt=-8.1388, Pc=-8.1388, Pd=-0.0
Ctrl: 0=1581; 1=1374; 
EVA: Load A from Aest.csv.


In [10]:
sse.neo

2866

In [11]:
# --- Response ---

# sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
#              lr=0.1, lp=40, seed=2022, name="EVA")
# sse.load_A("Aest.csv")

print(sse.neo)

x0 = sse.x0.copy()
print(sse.Prc)
sse.run(tf=18 + 300/3600, Pi=100,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)
print(sse.Prc)
x1 = sse.x0.copy()

2866
0


EVA MCS: 100%|██████████| 300/300 [00:55<00:00,  5.36it/s]

25.10274856028247


In [ ]:
sse.ev[sse.ev['ne'] >= sse.ev['nem']]

In [15]:
sse.ev[['lc', 'na', 'nam']]

,lc,na,nam
0,0.0,2439,2756
1,0.0,1480,3484
2,0.0,2707,2899
3,0.0,2537,2874
4,0.0,1607,3448
...,...,...,...
3508,0.0,2157,2856
3509,1.0,2154,2152
3510,0.0,2567,2996
3511,1.0,2121,2121


In [ ]:
sse.g_c(100)

In [13]:
sse.neo

2300.0

In [ ]:
# sse.r_agc(100)

In [ ]:
sse.ev.lc.sum()

In [ ]:
sse.ne

In [ ]:
sse.g_u()

In [ ]:
sse.neo

In [ ]:
sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             is_report=False)

# historical data
ev_num = pd.read_csv("ev_num.csv")

# calculate the sf
k1 = ev_num['ne'][(ev_num['time'] >= sse.ts) & (ev_num['time'] <= sse.ts+1/12)].mean()
k0 = ev_num['ne'][ev_num['time'] >= sse.ts].iloc[0]
k = k1 / k0

# estiamte FRC
sse.g_frc(nea=sse.ne*k)

In [ ]:
## Run to get the EV numbers within a day

# import joblib
# from joblib import Parallel, delayed

# Nt = 10000
# def rne(time):
#     sse1 = ev_ssm(ts=time, N=Nt, step=1, tp=40,
#                   lr=0.1, lp=60, seed=2022, name="EVA",
#                   is_report=False)
#     return sse1.ne

# # interval by minutes, 24H
# rnel = Parallel(n_jobs=joblib.cpu_count())(delayed(rne)(i) for i in np.arange(1/60, 24, 1/60))

# ev_num = pd.DataFrame()
# ev_num['time'] = np.arange(1/60, 24, 1/60)
# ev_num['ne'] = rnel
# ev_num.to_csv("ev_num.csv", index=False)